<a href="https://colab.research.google.com/github/rufous86/spark_vs_pandas/blob/main/pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Скачаем необходимые наборы данных с kaggle  
инструкция по скачиванию данных в colab напрямую с kaggle ниже:  
https://www.kaggle.com/general/74235

In [5]:
# ! pip install -q kaggle
# from google.colab import drive
# drive.mount('/content/drive')
# ! mkdir ~/.kaggle
# ! cp '/content/drive/MyDrive/Colab Notebooks/kaggle.json' ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle competitions download -c 'riiid-test-answer-prediction'
# ! mkdir data
# ! unzip riiid-test-answer-prediction.zip -d data

In [2]:
import pandas as pd

df_train = pd.read_csv('data/train.csv',
                        dtype={'content_id': 'int16',
                               'content_type_id': 'int8',
                               'task_container_id': 'int16',
                               'user_answer': 'int8',
                               'answered_correctly': 'int8',
                               'prior_question_elapsed_time': 'float32'})


In [3]:
display(df_train.head()) # выведем первые несколько строк таблицы
display(df_train.info()) # выведем информацию о данных нашей таблицы
display(df_train.shape) # посмотрим на размер нашей таблицы

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31000000 entries, 0 to 30999999
Data columns (total 10 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          int64  
 1   timestamp                       int64  
 2   user_id                         int64  
 3   content_id                      int16  
 4   content_type_id                 int8   
 5   task_container_id               int16  
 6   user_answer                     int8   
 7   answered_correctly              int8   
 8   prior_question_elapsed_time     float32
 9   prior_question_had_explanation  object 
dtypes: float32(1), int16(2), int64(3), int8(3), object(1)
memory usage: 1.2+ GB


None

(31000000, 10)

Посмотрим, сколько в нашей таблице пустых значений

In [4]:
df_train.isna().sum()

row_id                                 0
timestamp                              0
user_id                                0
content_id                             0
content_type_id                        0
task_container_id                      0
user_answer                            0
answered_correctly                     0
prior_question_elapsed_time       722951
prior_question_had_explanation    120904
dtype: int64

Видим, что полностью заполнены значениями все столбцы, кроме prior_question_elapsed_time и prior_question_had_explanation. В обоих случаях отсутствует очень малая часть всех данных. Заполним пропущенные значения нулями

In [5]:
cols_to_fillna = ['prior_question_elapsed_time', 'prior_question_had_explanation']
df_train[cols_to_fillna] = df_train[cols_to_fillna].fillna(0)

В таблице преобладают данные численного типа. Исключение составляет столбец prior_question_had_explanation.  
Посмотрим, какие уникальные значения составляют этот столбец. Также глянем на баланс этих значений

In [6]:
df_train['prior_question_had_explanation'].value_counts(True)

True    0.885955
0       0.114045
Name: prior_question_had_explanation, dtype: float64

Видим, что в столбце присутствуют два значения True и 0 (часть из них до применения функции fillna была False, часть была NaN). Приведём столбец prior_question_had_explanation к целочисленному типу

In [7]:
df_train['prior_question_had_explanation'] = df_train['prior_question_had_explanation'].astype('int8')

In [8]:
df_train.isna().sum()

row_id                            0
timestamp                         0
user_id                           0
content_id                        0
content_type_id                   0
task_container_id                 0
user_answer                       0
answered_correctly                0
prior_question_elapsed_time       0
prior_question_had_explanation    0
dtype: int64

Проанализируем характеристики, влияющие на успеваемость студентов. Так как фактически данные об успеваемости у нас отсутствуют, условно за успеваемость будут выступать правильно данные ответы.      
Сначала сохраним колонку answered_correctly в переменную target. Это будет наша целевая переменная  
Затем рассчитаем коэффициент корреляции целевой переменной с каждой из остальных характеристик

In [9]:
target = df_train.pop('answered_correctly')
df_train.corrwith(target)

row_id                           -0.000530
timestamp                         0.023392
user_id                          -0.000533
content_id                       -0.197708
content_type_id                  -0.437686
task_container_id                 0.060067
user_answer                       0.126696
prior_question_elapsed_time       0.068712
prior_question_had_explanation    0.257143
dtype: float64

Видны сильная прямая зависимость target со столбцом prior_question_had_explanation и сильные обратные зависимость со столбцами content_id и content_type_id

Еще раз глянем на колонку content_type_id  
В наличии 2 типа содержимого: программы и лекции. Попробуем из совместить с данными из таблиц questions.csv и lectures.csv

In [10]:
df_train['content_type_id'].value_counts(True)

0    0.980579
1    0.019421
Name: content_type_id, dtype: float64

Видим, в данном столбце храниться 2 значения. Проверим, если ли похожие данные в других таблицах

---

Посмотрим, что из себя представляет таблица questions.csv

In [11]:
df_questions = pd.read_csv('data/questions.csv')
display(df_questions.head())
display(df_questions.info())

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13523 entries, 0 to 13522
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_id     13523 non-null  int64 
 1   bundle_id       13523 non-null  int64 
 2   correct_answer  13523 non-null  int64 
 3   part            13523 non-null  int64 
 4   tags            13522 non-null  object
dtypes: int64(4), object(1)
memory usage: 528.4+ KB


None

Проверим, есть ли в таблице train.csv вопросы, отсутствующие в таблице questions.csv

In [12]:
content_id_unique = set(df_train[df_train['content_type_id'] == 0]['content_id'].unique())
question_id_unique = set(df_questions['question_id'].unique())
print(f'content_id_unique - question_id_unique = {len(content_id_unique - question_id_unique)}')

content_id_unique - question_id_unique = 0


Объединим таблицы train.csv и question.csv

In [13]:
df_train = df_train.merge(
    df_questions.rename({'question_id':'content_id'}, axis=1),
    how='left',
    on='content_id')
display(df_train.head())
display(df_train.info())

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,correct_answer,part,tags
0,0,0,115,5692,0,1,3,0.0,0,5692.0,3.0,5.0,151
1,1,56943,115,5716,0,2,2,37000.0,0,5716.0,2.0,5.0,168
2,2,118363,115,128,0,0,0,55000.0,0,128.0,0.0,1.0,131 149 92
3,3,131167,115,7860,0,3,0,19000.0,0,7860.0,0.0,1.0,131 104 81
4,4,137965,115,7922,0,4,1,11000.0,0,7922.0,1.0,1.0,131 149 92


<class 'pandas.core.frame.DataFrame'>
Int64Index: 31000000 entries, 0 to 30999999
Data columns (total 13 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          int64  
 1   timestamp                       int64  
 2   user_id                         int64  
 3   content_id                      int16  
 4   content_type_id                 int8   
 5   task_container_id               int16  
 6   user_answer                     int8   
 7   prior_question_elapsed_time     float32
 8   prior_question_had_explanation  int8   
 9   bundle_id                       float64
 10  correct_answer                  float64
 11  part                            float64
 12  tags                            object 
dtypes: float32(1), float64(3), int16(2), int64(3), int8(3), object(1)
memory usage: 2.2+ GB


None

In [14]:
df_train.isna().sum()

row_id                                 0
timestamp                              0
user_id                                0
content_id                             0
content_type_id                        0
task_container_id                      0
user_answer                            0
prior_question_elapsed_time            0
prior_question_had_explanation         0
bundle_id                         366100
correct_answer                    366100
part                              366100
tags                              366100
dtype: int64

---

Посмотрим, что из себя представляет таблица lectures.csv

In [15]:
df_lectures = pd.read_csv('data/lectures.csv')
display(df_lectures.head())
display(df_lectures.info())

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lecture_id  418 non-null    int64 
 1   tag         418 non-null    int64 
 2   part        418 non-null    int64 
 3   type_of     418 non-null    object
dtypes: int64(3), object(1)
memory usage: 13.2+ KB


None

Аналогично таблице questions.csv проверим, есть ли в таблице train.csv вопросы, отсутствующие в таблице lectures.csv

In [16]:
content_id_unique = set(df_train[df_train['content_type_id'] == 1]['content_id'].unique())
lecture_id_unique = set(df_lectures['lecture_id'].unique())
print(f'content_id_unique - lecture_id_unique = {len(content_id_unique - lecture_id_unique)}')

content_id_unique - lecture_id_unique = 0


Объединим таблицы train.csv и lectures.csv

In [17]:
df_train = df_train.merge(
    df_lectures.rename({'lecture_id':'content_id'}, axis=1),
    how='left',
    on='content_id')
display(df_train.head())
display(df_train.info())

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,correct_answer,part_x,tags,tag,part_y,type_of
0,0,0,115,5692,0,1,3,0.0,0,5692.0,3.0,5.0,151,NaN,NaN,NaN
1,1,56943,115,5716,0,2,2,37000.0,0,5716.0,2.0,5.0,168,NaN,NaN,NaN
2,2,118363,115,128,0,0,0,55000.0,0,128.0,0.0,1.0,131 149 92,NaN,NaN,NaN
3,3,131167,115,7860,0,3,0,19000.0,0,7860.0,0.0,1.0,131 104 81,NaN,NaN,NaN
4,4,137965,115,7922,0,4,1,11000.0,0,7922.0,1.0,1.0,131 149 92,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 31000000 entries, 0 to 30999999
Data columns (total 16 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   row_id                          int64  
 1   timestamp                       int64  
 2   user_id                         int64  
 3   content_id                      int16  
 4   content_type_id                 int8   
 5   task_container_id               int16  
 6   user_answer                     int8   
 7   prior_question_elapsed_time     float32
 8   prior_question_had_explanation  int8   
 9   bundle_id                       float64
 10  correct_answer                  float64
 11  part_x                          float64
 12  tags                            object 
 13  tag                             float64
 14  part_y                          float64
 15  type_of                         object 
dtypes: float32(1), float64(5), int16(2), int64(3), int8(3), object(2)
memo

None

Заполним все пропущенные значения нулями

In [18]:
df_train = df_train.fillna(0)

In [19]:
df_train.isna().sum()

row_id                            0
timestamp                         0
user_id                           0
content_id                        0
content_type_id                   0
task_container_id                 0
user_answer                       0
prior_question_elapsed_time       0
prior_question_had_explanation    0
bundle_id                         0
correct_answer                    0
part_x                            0
tags                              0
tag                               0
part_y                            0
type_of                           0
dtype: int64

Снова рассчитаем коэффициент корреляции целевой переменной с каждой из остальных характеристик

In [20]:
df_train.corrwith(target)

row_id                           -0.000530
timestamp                         0.023392
user_id                          -0.000533
content_id                       -0.197708
content_type_id                  -0.437686
task_container_id                 0.060067
user_answer                       0.126696
prior_question_elapsed_time       0.068712
prior_question_had_explanation    0.257143
bundle_id                         0.025013
correct_answer                    0.031111
part_x                            0.022671
tag                              -0.288897
part_y                           -0.296527
dtype: float64

По-прежнему сохраняется сильная прямая корреляция данных студентами правильных ответов с колонкой prior_question_had_explanation (были ли даны объяснения на решения предыдущих вопросов)  
Самая сильная корреляция, причем зависимость обратная, с колонкой content_type_id (дан вопрос или лекция) и соответствующий этому материалу id.  
Появились еще две очень сильные обратные зависимость целевой переменной с данными из таблицы lectures.csv. Одна с колонкой tag (тэг, соответствующий данной лекции). Другая - со столбцом part_y (категория данной лекции)